In [138]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn import linear_model

In [163]:
data = pd.read_csv('data_v1.csv')
name = data.columns.values
name = np.array(['gvkey', 'date', 'fyear', 'fquarter', 'industry_format', 'consol',
       'popsrc', 'datafmt', 'ticker', 'cusip', 'conpany_name', 'currency', 'calendaryr_quarter',
       'fiscalyear_qtr', 'total_asset', 'cash', 'st_debt', 'lt_debt', 'net_income', 'ebitda',
       'int_exp', 'exchange', 'status', 'mkt_val', 'group', 'industry', 'sector',
       'subindustry'])

In [164]:
data.columns = name
data.head()

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
0,1004,19900228,1989,3,INDL,C,D,STD,AIR,361105,...,6.109,14.251,2.758,11,A,NaN,2010.0,201010.0,20.0,20101010.0
1,1004,19900531,1989,4,INDL,C,D,STD,AIR,361105,...,6.224,13.137,2.309,11,A,NaN,2010.0,201010.0,20.0,20101010.0
2,1004,19900831,1990,1,INDL,C,D,STD,AIR,361105,...,6.697,15.400,2.607,11,A,NaN,2010.0,201010.0,20.0,20101010.0
3,1004,19901130,1990,2,INDL,C,D,STD,AIR,361105,...,0.126,8.871,2.708,11,A,NaN,2010.0,201010.0,20.0,20101010.0
4,1004,19910228,1990,3,INDL,C,D,STD,AIR,361105,...,3.977,11.344,2.587,11,A,NaN,2010.0,201010.0,20.0,20101010.0


In [165]:
# there is no dual-listing
a = data[['ticker','exchange']].drop_duplicates().groupby('ticker').count()
a[a['exchange'] != 1]

,exchange
ticker,


In [166]:
# how many company compustst is inactively updating the data
# when compustat stop track the data, we believe the company will not be included in the top 1000 market cap range.
# so not affecting our analysis
len(data[data['status']== 'I']['ticker'].unique())

8536

In [167]:
len(data[data['status']== 'A']['ticker'].unique())

5576

In [168]:
# check if any company have 2 status
# if a company is not updated by the database anymore,
# the status changed to I for each time period before

inactive = data[data['status']== 'I']['ticker'].unique()
active = data[data['status']== 'A']['ticker'].unique()
duplicates = []
for ticker in inactive:
    if ticker in active:
        duplicates.append(inactive[i])
duplicates


[]

In [169]:
# 0(n)
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
   
print(intersection(inactive, active)) 

[]


In [170]:
len(data['ticker'].unique())

14111

In [171]:
data[data['ticker']=='BAM'].tail()

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
20258,2369,20180930,2018,3,INDL,C,D,STD,BAM,112585104,...,163.0,3010.0,1274.0,11,A,42698.7038,4020.0,402030.0,40.0,40203010.0
20259,2369,20181231,2018,4,INDL,C,D,STD,BAM,112585104,...,1884.0,3883.0,1477.0,11,A,36629.7341,4020.0,402030.0,40.0,40203010.0
20260,2369,20190331,2019,1,INDL,C,D,STD,BAM,112585104,...,615.0,3629.0,1616.0,11,A,44565.3048,4020.0,402030.0,40.0,40203010.0
20261,2369,20190630,2019,2,INDL,C,D,STD,BAM,112585104,...,399.0,4405.0,1833.0,11,A,45671.2297,4020.0,402030.0,40.0,40203010.0
20262,2369,20190930,2019,3,INDL,C,D,STD,BAM,112585104,...,947.0,3993.0,1926.0,11,A,53474.1592,4020.0,402030.0,40.0,40203010.0


In [172]:
# make sure data is sorted by ticker and time ascendingly
#data.sort_values(by = ['ticker','date'],ascending = True, inplace = True)

In [173]:
data.tail()

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
667110,330227,20190331,2019,2,INDL,C,D,STD,CTRM,Y1146L109,...,0.056,0.225,NaN,14,A,13.1282,2030.0,203030.0,20.0,20303010.0
667111,330227,20190630,2019,3,INDL,C,D,STD,CTRM,Y1146L109,...,0.261,0.421,NaN,14,A,10.2480,2030.0,203030.0,20.0,20303010.0
667112,331856,20190331,2019,1,INDL,C,D,STD,IMUX,4525EP101,...,-4.313,-4.157,NaN,14,A,NaN,3520.0,352010.0,35.0,35201010.0
667113,331856,20190630,2019,2,INDL,C,D,STD,IMUX,4525EP101,...,-14.714,-6.783,NaN,14,A,114.5394,3520.0,352010.0,35.0,35201010.0
667114,331856,20190930,2019,3,INDL,C,D,STD,IMUX,4525EP101,...,-8.215,-8.068,NaN,14,A,100.7100,3520.0,352010.0,35.0,35201010.0


In [189]:
# ret is the table we will be used for analysis
ret = data[['fiscalyear_qtr', 'ticker', 'conpany_name','calendaryr_quarter', 'total_asset', 'st_debt', 'lt_debt', 'net_income', 'ebitda',
       'int_exp', 'exchange', 'status', 'mkt_val','industry', 'sector']].copy()
ret['ROA'] = ret['net_income']/ret['total_asset']
ret['total_debt'] = ret['st_debt']+ret['lt_debt']
ret['D/A'] = ret['total_debt']+ret['total_asset']

In [190]:
data[data['ticker']=='BAM'].tail()

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
20258,2369,20180930,2018,3,INDL,C,D,STD,BAM,112585104,...,163.0,3010.0,1274.0,11,A,42698.7038,4020.0,402030.0,40.0,40203010.0
20259,2369,20181231,2018,4,INDL,C,D,STD,BAM,112585104,...,1884.0,3883.0,1477.0,11,A,36629.7341,4020.0,402030.0,40.0,40203010.0
20260,2369,20190331,2019,1,INDL,C,D,STD,BAM,112585104,...,615.0,3629.0,1616.0,11,A,44565.3048,4020.0,402030.0,40.0,40203010.0
20261,2369,20190630,2019,2,INDL,C,D,STD,BAM,112585104,...,399.0,4405.0,1833.0,11,A,45671.2297,4020.0,402030.0,40.0,40203010.0
20262,2369,20190930,2019,3,INDL,C,D,STD,BAM,112585104,...,947.0,3993.0,1926.0,11,A,53474.1592,4020.0,402030.0,40.0,40203010.0


In [191]:
# match forward 1year ROA to current leverage
ret['lead_ROA'] = ret.groupby(['ticker'])['ROA'].shift(-4)

In [192]:
ret.tail()

,fiscalyear_qtr,ticker,conpany_name,calendaryr_quarter,total_asset,st_debt,lt_debt,net_income,ebitda,int_exp,exchange,status,mkt_val,industry,sector,ROA,total_debt,D/A,lead_ROA
667110,2019Q2,CTRM,CASTOR MARITIME INC,2019Q1,10.242,0.0,0.0,0.056,0.225,NaN,14,A,13.1282,203030.0,20.0,0.005468,0.0,10.242,NaN
667111,2019Q3,CTRM,CASTOR MARITIME INC,2019Q2,10.360,0.0,0.0,0.261,0.421,NaN,14,A,10.2480,203030.0,20.0,0.025193,0.0,10.360,NaN
667112,2019Q1,IMUX,IMMUNIC INC,2019Q1,NaN,NaN,NaN,-4.313,-4.157,NaN,14,A,NaN,352010.0,35.0,NaN,NaN,NaN,NaN
667113,2019Q2,IMUX,IMMUNIC INC,2019Q2,72.574,NaN,NaN,-14.714,-6.783,NaN,14,A,114.5394,352010.0,35.0,-0.202745,NaN,NaN,NaN
667114,2019Q3,IMUX,IMMUNIC INC,2019Q3,67.604,NaN,NaN,-8.215,-8.068,NaN,14,A,100.7100,352010.0,35.0,-0.121516,NaN,NaN,NaN


In [193]:
# calculate change in current leverage
ret['1q_before_D/A'] = ret.groupby(['ticker'])['D/A'].shift(1)
ret.tail()

,fiscalyear_qtr,ticker,conpany_name,calendaryr_quarter,total_asset,st_debt,lt_debt,net_income,ebitda,int_exp,exchange,status,mkt_val,industry,sector,ROA,total_debt,D/A,lead_ROA,1q_before_D/A
667110,2019Q2,CTRM,CASTOR MARITIME INC,2019Q1,10.242,0.0,0.0,0.056,0.225,NaN,14,A,13.1282,203030.0,20.0,0.005468,0.0,10.242,NaN,10.184
667111,2019Q3,CTRM,CASTOR MARITIME INC,2019Q2,10.360,0.0,0.0,0.261,0.421,NaN,14,A,10.2480,203030.0,20.0,0.025193,0.0,10.360,NaN,10.242
667112,2019Q1,IMUX,IMMUNIC INC,2019Q1,NaN,NaN,NaN,-4.313,-4.157,NaN,14,A,NaN,352010.0,35.0,NaN,NaN,NaN,NaN,NaN
667113,2019Q2,IMUX,IMMUNIC INC,2019Q2,72.574,NaN,NaN,-14.714,-6.783,NaN,14,A,114.5394,352010.0,35.0,-0.202745,NaN,NaN,NaN,NaN
667114,2019Q3,IMUX,IMMUNIC INC,2019Q3,67.604,NaN,NaN,-8.215,-8.068,NaN,14,A,100.7100,352010.0,35.0,-0.121516,NaN,NaN,NaN,NaN


In [194]:
ret['change_D/A']=(ret['D/A']-ret['1q_before_D/A'])/ret['1q_before_D/A']

In [196]:
ret[['fiscalyear_qtr', 'ticker','ROA', 'change_D/A', 'mkt_val']].head()

,fiscalyear_qtr,ticker,ROA,change_D/A,mkt_val
0,1989Q3,AIR,0.015918,NaN,NaN
1,1989Q4,AIR,0.016020,0.011707,NaN
2,1990Q1,AIR,0.017361,-0.019581,NaN
3,1990Q2,AIR,0.000329,0.006856,NaN
4,1990Q3,AIR,0.010374,-0.050173,NaN


In [202]:
# get rid of NAs
ret2=ret[['fiscalyear_qtr', 'ticker','ROA', 'change_D/A']].copy()
print(ret2)
ret2=ret2.dropna()
ret2.head()

       fiscalyear_qtr ticker       ROA  change_D/A
0              1989Q3    AIR  0.015918         NaN
1              1989Q4    AIR  0.016020    0.011707
2              1990Q1    AIR  0.017361   -0.019581
3              1990Q2    AIR  0.000329    0.006856
4              1990Q3    AIR  0.010374   -0.050173
5              1990Q4    AIR  0.010530    0.003452
6              1991Q1    AIR  0.009145    0.002061
7              1991Q2    AIR  0.006868    0.048792
8              1991Q3    AIR  0.007255    0.022925
9              1991Q4    AIR  0.002555   -0.025301
10             1992Q1    AIR  0.007779    0.022363
11             1992Q2    AIR  0.003936    0.014351
12             1992Q3    AIR -0.015252   -0.060915
13             1992Q4    AIR  0.003588   -0.038867
14             1993Q1    AIR  0.006635    0.014796
15             1993Q2    AIR  0.005825    0.133008
16             1993Q3    AIR  0.005441   -0.003630
17             1993Q4    AIR  0.005776    0.021010
18             1994Q1    AIR  0

,fiscalyear_qtr,ticker,ROA,change_D/A
1,1989Q4,AIR,0.016020,0.011707
2,1990Q1,AIR,0.017361,-0.019581
3,1990Q2,AIR,0.000329,0.006856
4,1990Q3,AIR,0.010374,-0.050173
5,1990Q4,AIR,0.010530,0.003452


In [204]:
# select the top 1000 market cap company for each quarter
#ret2['rank']=ret2.groupby(['fiscalyear_qtr'])['mkt_val'].rank(ascending = False)

In [205]:
dates = ret2['fiscalyear_qtr'].unique()
dates

array(['1989Q4', '1990Q1', '1990Q2', '1990Q3', '1990Q4', '1991Q1',
       '1991Q2', '1991Q3', '1991Q4', '1992Q1', '1992Q2', '1992Q3',
       '1992Q4', '1993Q1', '1993Q2', '1993Q3', '1993Q4', '1994Q1',
       '1994Q2', '1994Q3', '1994Q4', '1995Q1', '1995Q2', '1995Q3',
       '1995Q4', '1996Q1', '1996Q2', '1996Q3', '1996Q4', '1997Q1',
       '1997Q2', '1997Q3', '1997Q4', '1998Q1', '1998Q2', '1998Q3',
       '1998Q4', '1999Q1', '1999Q2', '1999Q3', '1999Q4', '2000Q1',
       '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2', '2001Q3',
       '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4', '2003Q1',
       '2003Q2', '2003Q3', '2003Q4', '2004Q1', '2004Q2', '2004Q3',
       '2004Q4', '2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1',
       '2006Q2', '2006Q3', '2006Q4', '2007Q1', '2007Q2', '2007Q3',
       '2007Q4', '2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1',
       '2009Q2', '2009Q3', '2009Q4', '2010Q1', '2010Q2', '2010Q3',
       '2010Q4', '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012

In [207]:
for date in dates:
    subdata = ret2[ret2['fiscalyear_qtr']==date]

In [208]:
def solve_sv_regression(X, Y):

    model = sm.OLS(Y, X)
    
    results_iid = model.fit() # Base case: assuming iid errors
    results_hetero = model.fit(cov_type='HC3') # Adjusted for heteroskedasticity
    results_serial = model.fit(cov_type='HAC',cov_kwds={'maxlags':6, 'use_correction': True})  # adjust for serial correlation
    
    summary = []
    summary.extend((results_iid.params[0],
                    results_iid.bse[0],
                    results_iid.tvalues[0],
                    results_hetero.bse[0],
                    results_hetero.tvalues[0],
                    results_serial.bse[0],
                    results_serial.tvalues[0]))
    return summary
   

In [211]:
solve_sv_regression(ret2[ret2['fiscalyear_qtr']=='2008Q1']['ROA'], ret2[ret2['fiscalyear_qtr']=='2008Q1']['change_D/A'])

[0.005041931029899615,
 0.6347600745336816,
 0.007943050031310817,
 0.014168275879261455,
 0.3558605911450134,
 0.013453255282707167,
 0.37477405460227314]

In [246]:
# 'ROA','change_D/A',
ret2[ret2['fiscalyear_qtr']=='1992Q2']

,fiscalyear_qtr,ticker,ROA,change_D/A
11,1992Q2,AIR,0.003936,0.014351
129,1992Q2,ACSE,-0.056874,-0.023170
151,1992Q2,ADCT.1,0.026249,-0.055276
236,1992Q2,AELNA,-0.001653,-0.012521
259,1992Q2,ALO.2,0.012771,-0.002767
334,1992Q2,UDI.,0.009750,-0.044833
378,1992Q2,AAL,-0.010301,0.027886
616,1992Q2,ASKI,-0.175919,-0.124193
633,1992Q2,ASTA,0.032342,0.035970
660,1992Q2,ARXX,0.003557,-0.223657


In [240]:
a[a['ticker']==True]

,ticker,ROA,change_D/A


In [216]:
solve_sv_regression(ret2[ret2['fiscalyear_qtr']=='1992Q2']['ROA'], ret2[ret2['fiscalyear_qtr']=='1992Q2']['change_D/A'])

[nan, nan, nan, nan, nan, nan, nan]

In [214]:
dates = ret2['fiscalyear_qtr'].unique()
dates

array(['1989Q4', '1990Q1', '1990Q2', '1990Q3', '1990Q4', '1991Q1',
       '1991Q2', '1991Q3', '1991Q4', '1992Q1', '1992Q2', '1992Q3',
       '1992Q4', '1993Q1', '1993Q2', '1993Q3', '1993Q4', '1994Q1',
       '1994Q2', '1994Q3', '1994Q4', '1995Q1', '1995Q2', '1995Q3',
       '1995Q4', '1996Q1', '1996Q2', '1996Q3', '1996Q4', '1997Q1',
       '1997Q2', '1997Q3', '1997Q4', '1998Q1', '1998Q2', '1998Q3',
       '1998Q4', '1999Q1', '1999Q2', '1999Q3', '1999Q4', '2000Q1',
       '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2', '2001Q3',
       '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4', '2003Q1',
       '2003Q2', '2003Q3', '2003Q4', '2004Q1', '2004Q2', '2004Q3',
       '2004Q4', '2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1',
       '2006Q2', '2006Q3', '2006Q4', '2007Q1', '2007Q2', '2007Q3',
       '2007Q4', '2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1',
       '2009Q2', '2009Q3', '2009Q4', '2010Q1', '2010Q2', '2010Q3',
       '2010Q4', '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012

In [215]:
dates = ret2['fiscalyear_qtr'].unique()
for date in dates:
    print(date)
    Y = ret2[ret2['fiscalyear_qtr']==date]['ROA']
    X = ret2[ret2['fiscalyear_qtr']==date]['change_D/A']
    solve_sv_regression(Y,X)


1989Q4
1990Q1
1990Q2
1990Q3
1990Q4
1991Q1


MissingDataError: exog contains inf or nans